# Modern Robotics Ch 3.2 — Rotation Matrices: 3 Uses (Orientation • Frame-change • Rotation)

This notebook demonstrates the **three common uses** of a rotation matrix \(R\in SO(3)\):

1. **Represent an orientation** (how frame \{b\} sits inside \{s\})
2. **Change reference frame / coordinates** (same physical vector, different coordinates)
3. **Rotate a vector or a frame** (apply a rotation to geometry)

We’ll use a simple “object” (a 3D box) plus **space** frame \{s\} and **body** frame \{b\}.

> Notation: \(R_{sb}\) maps body-coordinates to space-coordinates:  
> \[ v_s = R_{sb}\, v_b \]
> and \(R_{bs} = R_{sb}^T\):  
> \[ v_b = R_{sb}^T\, v_s \]


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa

def Rx(t):
    c, s = np.cos(t), np.sin(t)
    return np.array([[1, 0, 0],
                     [0, c, -s],
                     [0, s,  c]])

def Ry(t):
    c, s = np.cos(t), np.sin(t)
    return np.array([[ c, 0, s],
                     [ 0, 1, 0],
                     [-s, 0, c]])

def Rz(t):
    c, s = np.cos(t), np.sin(t)
    return np.array([[c, -s, 0],
                     [s,  c, 0],
                     [0,  0, 1]])

def is_so3(R, atol=1e-8):
    return (np.allclose(R.T @ R, np.eye(3), atol=atol) and
            np.isclose(np.linalg.det(R), 1.0, atol=atol))

def set_axes_equal_3d(ax, lim=1.5):
    ax.set_xlim(-lim, lim)
    ax.set_ylim(-lim, lim)
    ax.set_zlim(-lim, lim)
    ax.set_box_aspect([1, 1, 1])

def draw_arrow(ax, v, color='k', label=None, lw=2.5):
    ax.quiver(0, 0, 0, v[0], v[1], v[2], color=color, linewidth=lw, arrow_length_ratio=0.12)
    if label:
        ax.text(v[0], v[1], v[2], label, color=color)

def draw_frame(ax, R, name='b', axis_len=1.0, colors=('tab:red','tab:green','tab:blue'), lw=3):
    axes = [R[:,0]*axis_len, R[:,1]*axis_len, R[:,2]*axis_len]
    labels = [f'{name}x', f'{name}y', f'{name}z']
    for v, c, lab in zip(axes, colors, labels):
        draw_arrow(ax, v, color=c, label=lab, lw=lw)

def rotate_points(R, P):
    return (R @ P.T).T


## Choose yaw / pitch / roll and build \(R_{sb}\)

We’ll use the common composition:

\[
R_{sb} = R_z(\text{yaw})\,R_y(\text{pitch})\,R_x(\text{roll})
\]

**Order matters** (matrix multiplication applies right-to-left).


In [ ]:
yaw_deg, pitch_deg, roll_deg = 40, 20, -10
yaw, pitch, roll = np.deg2rad([yaw_deg, pitch_deg, roll_deg])

R_sb = Rz(yaw) @ Ry(pitch) @ Rx(roll)
print("R_sb =\n", R_sb)
print("SO(3) check:", is_so3(R_sb))


## Use case 1 — Represent an orientation

The **orientation** of \{b\} relative to \{s\} is captured by \(R_{sb}\).

Key geometric fact:
- **Columns of \(R_{sb}\)** are the **body axes expressed in space**:
  - col 1 = \(\hat x_b\) expressed in \{s\}
  - col 2 = \(\hat y_b\) expressed in \{s\}
  - col 3 = \(\hat z_b\) expressed in \{s\}


In [ ]:
fig = plt.figure(figsize=(7,6))
ax = fig.add_subplot(111, projection='3d')

draw_frame(ax, np.eye(3), name='s', axis_len=1.0, colors=('gray','gray','gray'), lw=2)
draw_frame(ax, R_sb, name='b', axis_len=0.9)

ax.set_title(f"Use case 1: Orientation (R_sb)\n(yaw={yaw_deg}°, pitch={pitch_deg}°, roll={roll_deg}°)")
ax.set_xlabel("X"); ax.set_ylabel("Y"); ax.set_zlabel("Z")
set_axes_equal_3d(ax, lim=1.3)
plt.show()


## A simple “object”: a box attached to the body frame

We define the box corners in **body coordinates**.  
Then we express those corners in space by multiplying by \(R_{sb}\).


In [ ]:
L, W, H = 1.2, 0.5, 0.3
corners_b = np.array([
    [-L/2, -W/2, -H/2],
    [-L/2, -W/2,  H/2],
    [-L/2,  W/2, -H/2],
    [-L/2,  W/2,  H/2],
    [ L/2, -W/2, -H/2],
    [ L/2, -W/2,  H/2],
    [ L/2,  W/2, -H/2],
    [ L/2,  W/2,  H/2],
])

edges = [
    (0,1),(0,2),(1,3),(2,3),
    (4,5),(4,6),(5,7),(6,7),
    (0,4),(1,5),(2,6),(3,7)
]

corners_s = rotate_points(R_sb, corners_b)

fig = plt.figure(figsize=(7,6))
ax = fig.add_subplot(111, projection='3d')

for i,j in edges:
    p, q = corners_s[i], corners_s[j]
    ax.plot([p[0], q[0]], [p[1], q[1]], [p[2], q[2]], linewidth=2)

draw_frame(ax, np.eye(3), name='s', axis_len=1.0, colors=('gray','gray','gray'), lw=2)
draw_frame(ax, R_sb, name='b', axis_len=0.9)

ax.set_title("Box (defined in {b}) expressed in {s} using R_sb")
ax.set_xlabel("X"); ax.set_ylabel("Y"); ax.set_zlabel("Z")
set_axes_equal_3d(ax, lim=1.5)
plt.show()


## Use case 2 — Change reference frame (coordinates)

If a physical vector has coordinates \(v_b\) in \{b\} and \(v_s\) in \{s\}, then:

\[
v_s = R_{sb}\, v_b
\]
and
\[
v_b = R_{sb}^T\, v_s
\]

### Concrete example
In body coordinates, the **forward direction** is always \(v_b=[1,0,0]^T\).  
But in the world it becomes \(v_s = R_{sb}v_b\).


In [ ]:
v_b = np.array([1.0, 0.0, 0.0])
v_s = R_sb @ v_b
v_b_back = R_sb.T @ v_s

print("v_b (body coords)  =", v_b)
print("v_s (space coords) =", np.round(v_s, 6))
print("back to body       =", np.round(v_b_back, 6))


In [ ]:
fig = plt.figure(figsize=(7,6))
ax = fig.add_subplot(111, projection='3d')

draw_frame(ax, np.eye(3), name='s', axis_len=1.0, colors=('gray','gray','gray'), lw=2)
draw_frame(ax, R_sb, name='b', axis_len=0.9)

draw_arrow(ax, 1.2*v_s, color='tab:orange', label='forward in {s}', lw=4)

ax.set_title("Use case 2: Frame-change (v_s = R_sb v_b)")
ax.set_xlabel("X"); ax.set_ylabel("Y"); ax.set_zlabel("Z")
set_axes_equal_3d(ax, lim=1.4)
plt.show()


In [ ]:
# --- Use case 2: Frame-change, shown as TWO coordinate descriptions (side-by-side) ---

def nice_view(ax):
    ax.view_init(elev=20, azim=35)


v_b = np.array([1.0, 0.0, 0.0])   # "forward" in body coordinates
v_s = R_sb @ v_b                  # same physical vector, but coordinates in space

fig = plt.figure(figsize=(14,6))
ax1 = fig.add_subplot(121, projection='3d')
ax2 = fig.add_subplot(122, projection='3d')

# LEFT: show v_b in the BODY frame
draw_frame(ax1, np.eye(3), name='s', axis_len=1.0, colors=('gray','gray','gray'), lw=2)
draw_frame(ax1, R_sb,      name='b', axis_len=0.9)

# draw the vector along body x-axis direction (i.e., b_x) and label as v_b coordinates
draw_arrow(ax1, 1.2 * R_sb[:,0], color='tab:orange', label=r"$v_b=[1,0,0]^T$ (in $\{b\}$)", lw=4)

ax1.set_title(r"Same physical vector, described in $\{b\}$ (body coords)")
ax1.set_xlabel("X"); ax1.set_ylabel("Y"); ax1.set_zlabel("Z")
set_axes_equal_3d(ax1, lim=1.4); nice_view(ax1)

# RIGHT: show v_s in the SPACE frame
draw_frame(ax2, np.eye(3), name='s', axis_len=1.0, colors=('gray','gray','gray'), lw=2)
draw_frame(ax2, R_sb,      name='b', axis_len=0.9)

# draw the same physical vector, but now label its coordinates in space
draw_arrow(ax2, 1.2 * v_s, color='tab:orange', label=r"$v_s=R_{sb}v_b$ (in $\{s\}$)", lw=4)

ax2.set_title(r"Same physical vector, described in $\{s\}$ (space coords)")
ax2.set_xlabel("X"); ax2.set_ylabel("Y"); ax2.set_zlabel("Z")
set_axes_equal_3d(ax2, lim=1.4); nice_view(ax2)

plt.suptitle(r"Use case 2: Change reference frame (coordinates) — vector is SAME, numbers change", y=0.98)
plt.show()

print("v_b =", v_b)
print("v_s =", np.round(v_s, 6))
print("Note: v_s equals the first column of R_sb =", np.round(R_sb[:,0], 6))


## Use case 3 — Rotate a vector (active rotation)


Actively rotate a vector \(w\) by \(R_{sb}\):

$$
w_{\text{rot}} = R_{sb}\, w
$$

Rotation preserves length:

$$
\|w\| = \|R_{sb}w\|
$$



In [ ]:
w = np.array([0.3, 0.8, 0.1])
w_rot = R_sb @ w

print("w      =", w)
print("R w    =", np.round(w_rot, 6))
print("||w||  =", np.linalg.norm(w))
print("||Rw|| =", np.linalg.norm(w_rot))


In [ ]:
fig = plt.figure(figsize=(7,6))
ax = fig.add_subplot(111, projection='3d')

draw_frame(ax, np.eye(3), name='s', axis_len=1.0, colors=('gray','gray','gray'), lw=2)

draw_arrow(ax, w, color='black', label='w', lw=3)
draw_arrow(ax, w_rot, color='tab:purple', label='R w', lw=3)

ax.set_title("Use case 3: Active rotation (w_rot = R_sb @ w)")
ax.set_xlabel("X"); ax.set_ylabel("Y"); ax.set_zlabel("Z")
set_axes_equal_3d(ax, lim=1.4)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

w = np.array([0.3, 0.8, 0.1])

# total frames and split into 3 phases
n = 90
n1, n2, n3 = 30, 30, 30  # roll, pitch, yaw

fig = plt.figure(figsize=(6,5))
ax = fig.add_subplot(111, projection="3d")

def draw_scene(R_current, title):
    ax.cla()
    draw_frame(ax, np.eye(3), name='s', axis_len=1.0, colors=('gray','gray','gray'), lw=2)
    w_rot = R_current @ w
    draw_arrow(ax, w, color="black", label="w", lw=3)
    draw_arrow(ax, w_rot, color="tab:blue", label="R w", lw=3)
    ax.set_title(title)
    ax.set_xlabel("X"); ax.set_ylabel("Y"); ax.set_zlabel("Z")
    set_axes_equal_3d(ax, lim=1.4)
    # nice_view(ax)  # if you have it

def update(k):
    # phase 1: roll about x
    if k < n1:
        a = (k/(n1-1)) * roll
        R = Rx(a)
        title = r"Active rotation: phase 1 (roll)  $R=R_x(\alpha)$"
    # phase 2: then pitch about y
    elif k < n1+n2:
        b = ((k-n1)/(n2-1)) * pitch
        R = Ry(b) @ Rx(roll)
        title = r"phase 2 (pitch)  $R=R_y(\beta)R_x(\mathrm{roll})$"
    # phase 3: then yaw about z
    else:
        g = ((k-n1-n2)/(n3-1)) * yaw
        R = Rz(g) @ Ry(pitch) @ Rx(roll)
        title = r"phase 3 (yaw)  $R=R_z(\gamma)R_y(\mathrm{pitch})R_x(\mathrm{roll})$"
    draw_scene(R, title)

from pathlib import Path
# Set repo root (edit this line if your notebook is elsewhere)\
ROOT = Path(r"/home/villy/code/modern-robotics-labbook/")
ASSETS = ROOT / "assets"
ASSETS.mkdir(parents=True, exist_ok=True)

ani = FuncAnimation(fig, update, frames=n, interval=50)
ani.save(str(ASSETS / "usecase3_active_rotation.gif"), writer="pillow", fps=20)
plt.close(fig)

print(f"Saved to: {(ASSETS / 'usecase3_active_rotation.gif').resolve()}")


## Summary

- **Use case 1 (orientation):** \(R_{sb}\) describes how \{b\} is oriented relative to \{s\}.  
- **Use case 2 (frame-change):** \(v_s = R_{sb} v_b\) converts coordinates from body → space.  
- **Use case 3 (rotate):** \(w_{rot} = R_{sb} w\) actively rotates a vector/shape, preserving length.

Next: add translation to move from **SO(3)** to **SE(3)** (homogeneous transforms).
